In [1]:
import requests

response = requests.get("https://parallelum.com.br/fipe/api/v1/carros/marcas/59/modelos/5940/anos/2014-3")
if response.status_code == 200:
  print("yay")
  print(response.text)

yay
{"TipoVeiculo":1,"Valor":"R$ 98.194,00","Marca":"VW - VolksWagen","Modelo":"AMAROK High.CD 2.0 16V TDI 4x4 Dies. Aut","AnoModelo":2014,"Combustivel":"Diesel","CodigoFipe":"005340-6","MesReferencia":"novembro de 2024","SiglaCombustivel":"D"}


In [2]:
!pip install groq

In [3]:
from groq import Groq

In [4]:
client = Groq(
    api_key="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"
)

completion = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": "Help figure out what to wear today. It's raining lightly and it's around 10 degrees Celsius."
        },
        {
            "role": "user",
            "content": ""
        }
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    content = chunk.choices[0].delta.content
    if content:
        print(content, end="")

On a lightly rainy day with a temperature of around 10 degrees Celsius, it's essential to prioritize staying dry and warm. Here's a suggested outfit:

1. **Waterproof or water-repellent outerwear**: Choose a waterproof jacket or a water-repellent windbreaker to keep you dry. Look for something with a hood to protect your head from the rain.
2. **Insulating mid-layer**: Wear a lightweight, breathable insulating layer such as a fleece jacket or a sweater to keep you warm. This will help retain body heat without making you feel too bulky.
3. **Base layer**: Opt for a moisture-wicking base layer (top and bottom) to keep your skin dry and comfortable. This can be a thermal top and leggings or long johns.
4. **Pants**: Choose water-repellent pants or leggings to keep your lower half dry. You can also consider wearing trousers with a waterproof and breathable membrane, such as Gore-Tex or similar technology.
5. **Footwear**: Wear waterproof shoes or boots with good grip to keep your feet dry 